# MHAR-ReCov Spillover Analysis with LASSO on PIT-Transformed Realized Covariances



### Model Description

To analyze the role of covariances in the volatility transmission mechanism across electricity markets, we use the MHAR-ReCov model, as proposed by Fengler and Gisler (2015). This model incorporates realized variances and covariances to account for interconnectedness between markets driven by common informational shocks.

The model is specified as:

$$
\tilde{Y}_t = \tilde{\beta}_0 + \sum_{h \in \{1,7,30\}} \tilde{\beta}_h \tilde{Y}_{h,t-1} + \tilde{\epsilon}_t
$$

Where:

- $ \tilde{Y}_t = \text{vech}(\text{ReCov}_t) $ is the half-vectorized $K$-dimensional daily realized covariance matrix, computed from high-frequency data of $N$ electricity markets.
- $ K = \frac{N(N+1)}{2} $ is the number of unique elements in the symmetric covariance matrix.
- $ \tilde{\beta}_0 $ is a vector of intercepts of length $K$.
- $ \tilde{\beta}_h $ are $K \times K$ coefficient matrices for $h = 1$ (daily), $h = 7$ (weekly), and $h = 30$ (monthly) lags.
- $ \tilde{\epsilon}_t $ is a $K \times 1$ innovation vector with zero mean and finite variance.

### Estimation

- The model is estimated using **LASSO** (Least Absolute Shrinkage and Selection Operator), allowing for regularization and variable selection in high-dimensional settings.
- The estimation is done on the **full sample** of realized covariance matrices.




### Step 1: Select and Load PIT-transformed vech Dataset

We use previously computed PIT-transformed realized covariance matrices. Each row represents the half-vectorized lower triangle (vech) of the daily covariance matrix.

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
import numpy as np
from joblib import Parallel, delayed

In [3]:
# Select dataset from Europe or from Australia

option_selector = widgets.Dropdown(
    options=['europe', 'australia'],
    value='europe',
    description='Dataset:',
)
button = widgets.Button(description="Load Dataset", button_style='success', icon='check')

def on_button_click(b):
    global pit_vech
    clear_output(wait=True)
    display(option_selector, button)
    selected_option = option_selector.value
    file_path = f"parquet_files/pit_transformed_vech_{selected_option}.parquet"
    try:
        pit_vech = pd.read_parquet(file_path)
        print(f"✅ Loaded: {file_path}")
        display(pit_vech.head())
    except Exception as e:
        print(f"❌ Failed to load: {e}")

button.on_click(on_button_click)
display(option_selector, button)

Dropdown(description='Dataset:', index=1, options=('europe', 'australia'), value='australia')

Button(button_style='success', description='Load Dataset', icon='check', style=ButtonStyle())

,nsw,qld-nsw,qld,sa-nsw,sa-qld,sa,tas-nsw,tas-qld,tas-sa,tas,vic-nsw,vic-qld,vic-sa,vic-tas,vic
2009-07-01,-0.699212,-0.895466,-1.271926,-0.366649,-0.786238,-0.097437,-0.118865,-0.045462,-0.062529,-0.011362,-0.630993,-0.582337,-0.674936,-0.081039,-0.901846
2009-07-02,0.282052,0.749995,1.040695,0.096010,-1.273522,0.046172,0.167654,0.316959,0.379591,-0.024856,0.382645,-0.273933,0.376540,0.512218,0.442966
2009-07-03,-0.661619,-0.362096,-0.563926,-0.466590,-0.122441,-0.589925,-0.508174,0.031960,-0.089591,-0.683880,-0.229950,-0.092444,-0.199430,-0.108145,-0.319200
2009-07-04,-0.720222,-0.597548,-0.947581,-0.626666,-0.304291,-0.711049,-0.187131,-0.073205,-0.172697,-0.065375,-0.413391,-0.276146,-0.375016,-0.208130,-0.551482
2009-07-05,-0.212486,0.012072,-0.117435,-0.045462,0.282791,-0.153986,0.094583,0.596699,0.413391,-0.218300,0.137480,0.296120,0.167654,0.412618,0.083889


 ## Step 2: Construct MHAR Regressors (Lags at 1, 7, 30)

 We construct lagged versions of the PIT-transformed vector at lags 1, 7, and 30 days, then concatenate them to form the full MHAR regressor matrix.


In [4]:
# Create MHAR lags at horizons: 1, 7, 30
def create_mhar_lags(df, horizons=[1, 7, 30]):
    df_lagged = pd.concat([df.shift(h).add_suffix(f"_lag{h}") for h in horizons], axis=1)
    return pd.concat([df, df_lagged], axis=1).dropna()

mhar_data = create_mhar_lags(pit_vech)
X = mhar_data.drop(columns=pit_vech.columns)
Y = mhar_data[pit_vech.columns]


### Step 3: Fit LASSO Regressions

We estimate a separate LASSO regression for each component of the PIT-transformed vech using the MHAR lag structure as regressors.
This enforces sparsity and prevents overfitting when estimating high-dimensional systems.


In [5]:
alphas = np.logspace(-4, 1, 50)  

def fit_lasso_response(y, X):
    model = LassoCV(alphas=alphas, cv=5).fit(X, y)
    return model.coef_, model.intercept_

results = Parallel(n_jobs=-1)(delayed(fit_lasso_response)(Y[col], X) for col in Y.columns)

betas = np.array([r[0] for r in results])  # shape: [K, 3K]
intercepts = np.array([r[1] for r in results])


## Step 4: Compute Residual Covariance Matrix

After estimating the MHAR-LASSO model, we compute in-sample residuals and use them to estimate the residual covariance matrix needed for spillover analysis.


In [6]:
Y_hat = X @ betas.T + intercepts
residuals = Y.values - Y_hat
Sigma = np.cov(residuals.T)


### Step 5: Compute Generalized Variance Decomposition (GVD)

We compute the 1-step ahead GVD using generalized forecast error variance decomposition based on Diebold & Yilmaz (2012). This tells us how much of forecast error variance in variable *i* is explained by shocks to variable *j*.


In [7]:
K = Y.shape[1]
H = 1  # horizon

# Build MA(1) coefficients from MHAR
Phi_mhar = np.zeros((1, K, K))  # only 1 horizon
for i in range(K):
    Phi_mhar[0, i, :] = betas[i, :K] + betas[i, K:2*K] + betas[i, 2*K:]  # sum lags 1,7,30

# Compute A matrices
A = [np.eye(K)]
for h in range(1, H+1):
    A_h = np.zeros((K, K))
    for j in range(min(h, Phi_mhar.shape[0])):
        A_h += Phi_mhar[j] @ A[h - j - 1]
    A.append(A_h)

# GVD Computation
theta_g = np.zeros((K, K))
sigma_diag_inv = np.diag(1 / np.diag(Sigma))

for i in range(K):
    e_i = np.zeros(K)
    e_i[i] = 1
    denom = sum([e_i @ A[h] @ Sigma @ A[h].T @ e_i for h in range(H)])
    for j in range(K):
        e_j = np.zeros(K)
        e_j[j] = 1
        numer = sum([(e_i @ A[h] @ Sigma @ e_j)**2 for h in range(H)])
        theta_g[i, j] = sigma_diag_inv[j, j] * numer / denom

theta_g_normalized = theta_g / theta_g.sum(axis=1, keepdims=True)




 ## Step 6: Compute Directional and Net Spillovers

In [8]:
spillover_matrix = theta_g_normalized * 100
TSI = 100 * (np.sum(theta_g_normalized) - np.trace(theta_g_normalized)) / K


In [10]:
# TABLE
# Base setup
label_names = pit_vech.columns.tolist()
K = len(label_names)

# Convert normalized GVD matrix to percentages
spillover_matrix = theta_g_normalized * 100

# Directional spillovers
directional_to = spillover_matrix.sum(axis=0) - np.diag(spillover_matrix)
directional_from = spillover_matrix.sum(axis=1) - np.diag(spillover_matrix)
net_directional = directional_to - directional_from

# Construct spillover table
spillover_table = pd.DataFrame(spillover_matrix, index=label_names, columns=label_names)
spillover_table["Directional FROM others"] = directional_from
spillover_table.loc["Directional TO others"] = list(directional_to) + [directional_to.sum()]
spillover_table.loc["NET Directional"] = list(net_directional) + [np.nan]

# Total Spillover Index
TSI = 100 * (np.sum(theta_g_normalized) - np.trace(theta_g_normalized)) / K

# Display results
display(spillover_table.round(2))

# Caption
from IPython.display import Markdown
caption = f"""
**Table X**: Spillovers based on MHAR-ReCov using PIT-transformed realized covariances (vech format).

This table reports the estimated forecast error variance spillovers across markets for a 1-day horizon.
Each cell $(i, j)$ shows the contribution of shocks from variable $j$ to the forecast error variance of variable $i$.

**Total Spillover Index (TSI)** = **{TSI:.2f}%**
"""
display(Markdown(caption))



,nsw,qld-nsw,qld,sa-nsw,sa-qld,sa,tas-nsw,tas-qld,tas-sa,tas,vic-nsw,vic-qld,vic-sa,vic-tas,vic,Directional FROM others
nsw,27.63,7.48,11.69,10.69,1.39,11.74,3.11,0.79,1.09,5.73,3.84,1.58,2.70,2.28,8.25,72.37
qld-nsw,9.74,35.95,23.28,1.19,7.77,0.71,0.72,2.84,1.51,0.47,3.82,5.32,2.81,0.97,2.89,64.05
qld,15.59,23.86,36.86,1.86,2.46,1.59,0.67,1.17,0.88,0.90,3.25,3.00,2.07,1.09,4.74,63.14
sa-nsw,10.92,0.94,1.43,28.24,3.56,20.72,1.69,1.75,1.48,1.99,6.96,3.10,3.50,3.88,9.84,71.76
sa-qld,2.05,8.78,2.72,5.13,40.62,1.28,1.96,5.76,1.35,0.49,5.61,17.29,3.25,0.66,3.07,59.38
sa,13.39,0.63,1.36,23.12,0.99,31.51,1.01,0.54,0.92,2.32,3.98,1.01,3.26,2.98,12.99,68.49
tas-nsw,3.70,0.66,0.60,1.97,1.59,1.05,32.94,4.40,9.89,19.38,6.64,1.90,5.07,6.55,3.64,67.06
tas-qld,1.29,3.55,1.43,2.78,6.35,0.77,6.00,44.85,7.03,1.12,3.83,8.82,1.19,9.32,1.68,55.15
tas-sa,1.36,1.45,0.82,1.80,1.14,1.01,10.35,5.40,34.46,3.08,8.54,1.07,8.62,16.47,4.42,65.54
tas,8.08,0.51,0.96,2.74,0.47,2.87,22.93,0.97,3.48,38.96,3.79,0.72,2.86,4.47,6.19,61.04



**Table X**: Spillovers based on MHAR-ReCov using PIT-transformed realized covariances (vech format).

This table reports the estimated forecast error variance spillovers across markets for a 1-day horizon.
Each cell $(i, j)$ shows the contribution of shocks from variable $j$ to the forecast error variance of variable $i$.

**Total Spillover Index (TSI)** = **66.34%**


### Step 7: Bootstrap standard errors

In [11]:
pit_vech = pd.read_parquet(f"parquet_files/pit_transformed_vech_{option_selector.value}.parquet")

# Load labels
import json
with open(f"parquet_files/vech_labels_{option_selector.value}.json", "r") as f:
    vech_labels = json.load(f)
    
pit_vech.columns = vech_labels

In [12]:
# Settings
n_bootstrap = 1000
block_size = 10
H = 1
alphas = np.logspace(-4, 1, 50)
pit_vech.columns = vech_labels
label_names = pit_vech.columns.tolist()
K = len(label_names)
labels = pit_vech.columns
np.random.seed(42)

In [13]:
# Block bootstrap indices
def block_bootstrap_indices(n_obs, block_size):
    n_blocks = int(np.ceil(n_obs / block_size))
    block_starts = np.random.randint(0, n_obs - block_size + 1, size=n_blocks)
    indices = []
    for start in block_starts:
        indices.extend(range(start, start + block_size))
    return indices[:n_obs]


In [14]:
assert list(pit_vech.columns) == vech_labels, "Column order mismatch!"


In [15]:
# Bootstrap iteration

def single_bootstrap(seed):
    np.random.seed(seed)
    indices = block_bootstrap_indices(len(pit_vech), block_size)
    sample = pit_vech.iloc[indices]

    # MHAR lags
    def create_mhar_lags(df, horizons=[1, 7, 30]):
        df_lagged = pd.concat([df.shift(h).add_suffix(f"_lag{h}") for h in horizons], axis=1)
        return pd.concat([df, df_lagged], axis=1).dropna()

    data = create_mhar_lags(sample)
    X = data.drop(columns=pit_vech.columns)
    Y = data[pit_vech.columns]

    # Fit LASSO
    def fit_lasso(y):
        model = LassoCV(alphas=alphas, cv=5).fit(X, y)
        return model.coef_, model.intercept_

    results = [fit_lasso(Y[col]) for col in Y.columns]
    betas = np.array([r[0] for r in results])
    intercepts = np.array([r[1] for r in results])
    Y_hat = X @ betas.T + intercepts
    residuals = Y.values - Y_hat
    Sigma = np.cov(residuals.T)
    
    # Build Phi (sum of 3 blocks)
    Phi_mhar = np.zeros((1, K, K))
    for i in range(K):
        Phi_mhar[0, i, :] = betas[i, :K] + betas[i, K:2*K] + betas[i, 2*K:]

    A = [np.eye(K)]
    for h in range(1, H+1):
        A_h = np.zeros((K, K))
        for j in range(min(h, Phi_mhar.shape[0])):
            A_h += Phi_mhar[j] @ A[h - j - 1]
        A.append(A_h)

    # GVD
    theta_g = np.zeros((K, K))
    sigma_diag_inv = np.diag(1 / np.diag(Sigma))

    for i in range(K):
        e_i = np.zeros(K)
        e_i[i] = 1
        denom = sum([e_i @ A[h] @ Sigma @ A[h].T @ e_i for h in range(H)])
        for j in range(K):
            e_j = np.zeros(K)
            e_j[j] = 1
            numer = sum([(e_i @ A[h] @ Sigma @ e_j)**2 for h in range(H)])
            theta_g[i, j] = sigma_diag_inv[j, j] * numer / denom

    theta_g_normalized = theta_g / theta_g.sum(axis=1, keepdims=True)
    tsi = 100 * (np.sum(theta_g_normalized) - np.trace(theta_g_normalized)) / K
    return theta_g_normalized, tsi


In [16]:
# Run bootstrap
seeds = np.random.randint(0, 1_000_000, size=n_bootstrap)
results = Parallel(n_jobs=-1)(delayed(single_bootstrap)(s) for s in tqdm(seeds))

theta_g_bootstrap = np.array([r[0] for r in results])
TSI_bootstrap = np.array([r[1] for r in results])


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [05:24<00:00,  3.08it/s]


In [17]:
# Point estimates already from main run: theta_g_normalized
theta_g_mean = theta_g_bootstrap.mean(axis=0) * 100
theta_g_se = theta_g_bootstrap.std(axis=0) * 100

TSI_mean = TSI_bootstrap.mean()
TSI_se = TSI_bootstrap.std()

print(f"TSI = {TSI_mean:.2f} ± {TSI_se:.2f}")


TSI = 72.32 ± 1.03


### Step 8: Create and Save Spillover Table



In [18]:
# ---- CONFIGURATION ----
n_bootstrap = theta_g_bootstrap.shape[0]

# Generate vech-style labels: ['NSW', 'QLD', 'QLD-NSW', ...]
label_names = pit_vech.columns.tolist()
K = len(label_names)

spillover_table_bootstrap = pd.DataFrame(index=label_names, columns=label_names)

for i in range(K):
    for j in range(K):
        mean_val = theta_g_mean[i, j]
        se_val = theta_g_se[i, j]
        spillover_table_bootstrap.iloc[i, j] = f"{mean_val:.2f} ± {se_val:.2f}"

K_full = len(label_names)

# ---- COMPUTE MEANS AND STANDARD ERRORS ----
theta_g_mean = theta_g_bootstrap.mean(axis=0) * 100  # shape: (K, K)
theta_g_se = theta_g_bootstrap.std(axis=0) * 100     # shape: (K, K)

# ---- CREATE FORMATTED SPILLOVER TABLE ----
spillover_table_bootstrap = pd.DataFrame(index=label_names, columns=label_names)
for i in range(K):
    for j in range(K):
        mean_val = theta_g_mean[i, j]
        se_val = theta_g_se[i, j]
        spillover_table_bootstrap.iloc[i, j] = f"{mean_val:.2f} ± {se_val:.2f}"

# ---- DIRECTIONAL METRICS ----
directional_to = theta_g_mean.sum(axis=0) - np.diag(theta_g_mean)
directional_from = theta_g_mean.sum(axis=1) - np.diag(theta_g_mean)
net_directional = directional_to - directional_from

# Compute bootstrap SEs for directional components
dir_to_se = theta_g_bootstrap.sum(axis=2) - np.einsum("ijk->ij", theta_g_bootstrap)
dir_from_se = theta_g_bootstrap.sum(axis=1) - np.einsum("ijk->ik", theta_g_bootstrap)
net_se = dir_to_se - dir_from_se

dir_to_row = [f"{v:.2f} ± {s:.2f}" for v, s in zip(directional_to, dir_to_se.std(axis=0) * 100)]
dir_from_col = [f"{v:.2f} ± {s:.2f}" for v, s in zip(directional_from, dir_from_se.std(axis=0) * 100)]
net_row = [f"{v:.2f} ± {s:.2f}" for v, s in zip(net_directional, net_se.std(axis=0) * 100)]

# ---- APPEND ROWS AND COLUMNS ----
spillover_table_bootstrap["Directional FROM others"] = dir_from_col
spillover_table_bootstrap.loc["Directional TO others"] = dir_to_row + [""]
spillover_table_bootstrap.loc["NET Directional"] = net_row + [""]



In [19]:
# Display nicely in Jupyter (monospaced font, centered, bold headers)
styled = (
    spillover_table_bootstrap
    .style
    .set_properties(**{'text-align': 'center', 'font-family': 'monospace'})
    .set_table_styles([{'selector': 'th', 'props': [('font-weight', 'bold')]}])
    .set_caption("**Table X: Spillovers based on MHAR-ReCov (Bootstrap Estimates)**")
)

display(styled)


,nsw_nsw,qld_nsw,qld_qld,sa_nsw,sa_qld,sa_sa,tas_nsw,tas_qld,tas_sa,tas_tas,vic_nsw,vic_qld,vic_sa,vic_tas,vic_vic,Directional FROM others
nsw_nsw,23.58 ± 0.93,8.02 ± 0.68,11.46 ± 0.60,10.03 ± 0.65,2.18 ± 0.34,10.85 ± 0.57,3.71 ± 0.45,1.08 ± 0.20,1.40 ± 0.25,6.23 ± 0.38,4.62 ± 0.48,2.54 ± 0.32,3.47 ± 0.40,2.41 ± 0.31,8.40 ± 0.41,76.42 ± 0.00
qld_nsw,10.13 ± 0.79,29.80 ± 1.35,20.75 ± 0.90,2.15 ± 0.41,7.88 ± 0.71,1.69 ± 0.35,1.46 ± 0.33,2.93 ± 0.46,1.78 ± 0.33,1.41 ± 0.28,4.70 ± 0.47,6.18 ± 0.62,3.65 ± 0.44,1.31 ± 0.28,4.20 ± 0.50,70.20 ± 0.00
qld_qld,14.82 ± 0.71,21.24 ± 0.94,30.50 ± 1.22,2.82 ± 0.37,3.43 ± 0.40,2.65 ± 0.37,1.41 ± 0.28,1.43 ± 0.28,1.15 ± 0.21,2.01 ± 0.32,4.21 ± 0.40,4.23 ± 0.47,2.96 ± 0.32,1.32 ± 0.24,5.83 ± 0.52,69.50 ± 0.00
sa_nsw,9.99 ± 0.70,1.70 ± 0.34,2.17 ± 0.30,23.49 ± 0.94,4.39 ± 0.54,18.03 ± 0.68,2.61 ± 0.37,2.25 ± 0.30,2.06 ± 0.36,3.02 ± 0.30,7.50 ± 0.67,4.08 ± 0.47,4.55 ± 0.54,4.11 ± 0.39,10.05 ± 0.44,76.51 ± 0.00
sa_qld,2.98 ± 0.43,8.53 ± 0.76,3.62 ± 0.41,6.03 ± 0.66,32.29 ± 1.73,2.51 ± 0.39,2.90 ± 0.43,5.61 ± 0.68,1.86 ± 0.37,1.48 ± 0.24,6.46 ± 0.61,15.52 ± 1.00,4.38 ± 0.53,1.20 ± 0.29,4.65 ± 0.49,67.71 ± 0.00
sa_sa,11.98 ± 0.66,1.48 ± 0.31,2.26 ± 0.32,19.99 ± 0.80,2.03 ± 0.35,26.04 ± 0.99,1.98 ± 0.32,1.08 ± 0.20,1.58 ± 0.32,3.47 ± 0.33,5.22 ± 0.55,2.19 ± 0.29,4.54 ± 0.60,3.41 ± 0.37,12.76 ± 0.58,73.96 ± 0.00
tas_nsw,4.25 ± 0.51,1.33 ± 0.30,1.25 ± 0.25,3.00 ± 0.41,2.43 ± 0.39,2.06 ± 0.33,27.03 ± 1.25,4.67 ± 0.54,9.24 ± 0.65,17.20 ± 0.78,7.20 ± 0.55,2.84 ± 0.38,5.79 ± 0.54,6.71 ± 0.56,4.99 ± 0.42,72.97 ± 0.00
tas_qld,1.71 ± 0.29,3.68 ± 0.56,1.75 ± 0.33,3.58 ± 0.39,6.52 ± 0.76,1.55 ± 0.26,6.47 ± 0.68,37.51 ± 2.00,7.68 ± 0.68,1.87 ± 0.34,4.62 ± 0.47,8.57 ± 0.83,2.07 ± 0.35,9.69 ± 0.84,2.73 ± 0.38,62.49 ± 0.00
tas_sa,1.76 ± 0.30,1.76 ± 0.33,1.11 ± 0.19,2.59 ± 0.43,1.70 ± 0.35,1.79 ± 0.35,10.10 ± 0.73,6.06 ± 0.63,29.54 ± 1.42,3.81 ± 0.41,8.52 ± 0.59,1.76 ± 0.34,8.47 ± 0.73,15.85 ± 0.91,5.20 ± 0.44,70.46 ± 0.00
tas_tas,8.10 ± 0.50,1.45 ± 0.28,2.02 ± 0.32,3.94 ± 0.36,1.40 ± 0.24,4.08 ± 0.38,19.51 ± 0.95,1.53 ± 0.29,3.96 ± 0.44,30.65 ± 1.33,5.08 ± 0.43,1.85 ± 0.29,4.10 ± 0.37,4.76 ± 0.53,7.58 ± 0.48,69.35 ± 0.00


In [25]:
# Export to LaTeX 
latex_code = spillover_table_bootstrap.to_latex(index=True, escape=False, caption="Spillovers based on MHAR-ReCov (Bootstrap Estimates)", label="tab:spillovers_bootstrap")
with open("spillover_bootstrap_table.tex", "w") as f:
    f.write(latex_code)


In [ ]:
spillover_table_bootstrap.to_parquet(f"parquet_files/volatility_spillovers_VAR_fullcov_{option_selector.value}.parquet")
